# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
pip install gmaps

  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076101 sha256=17d460dbb036e6b2fac4f2cc2457cbe7ab8b65d2897f50eb60dd7afb0ab0bbce
  Stored in directory: C:\Users\jacob\AppData\Local\pip\Cache\wheels\33\eb\41\5302168c36fc8ce8f48143bb58fde1015d5bc0c62bc3ddbb38
Successfully built gmaps
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
city_data_df = pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,ID,City,Lat,Lng,Cloudiness,Wind Speed,Humidity,Max Temp,Country,Date
0,0,georgetown,5.41,100.34,20,2.24,94,78.80,MY,1595202546
1,1,les cayes,18.20,-73.75,100,5.44,75,80.91,HT,1595202546
2,2,zyryanka,65.75,150.85,10,15.70,41,75.40,RU,1595202546
3,3,bluff,-46.60,168.33,75,3.00,83,39.00,NZ,1595202546
4,4,wajir,1.75,40.06,85,17.20,68,72.64,KE,1595202546


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
gmaps.configure(api_key=g_key)


In [25]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locs, weights=humidity, dissipating=False, max_intensity=300,point_radius=5)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
new_city_df = city_data_df.loc[(city_data_df["Max Temp"]<80) & (city_data_df["Max Temp"]>70) \
                              &(city_data_df["Wind Speed"]<10) \
                              &(city_data_df["Cloudiness"] == 0)].dropna()
new_city_df

,ID,City,Lat,Lng,Cloudiness,Wind Speed,Humidity,Max Temp,Country,Date
21,21,ajdabiya,30.76,20.23,0,9.66,70,72.10,LY,1595202548
68,68,boysun,38.21,67.20,0,4.07,22,73.60,UZ,1595202555
85,85,surt,31.21,16.59,0,9.55,78,76.26,LY,1595202558
95,95,kallithea,37.95,23.70,0,3.56,64,79.00,GR,1595202559
103,103,rio brilhante,-21.80,-54.55,0,9.53,48,71.08,BR,1595202560
118,118,gornoye loo,43.71,39.60,0,2.24,88,73.40,RU,1595202562
151,151,manavgat,36.79,31.44,0,1.68,88,77.00,TR,1595202567
228,228,marsa matruh,31.35,27.25,0,7.76,80,74.97,EG,1595202599
283,283,hyeres,43.12,6.12,0,6.42,77,73.99,FR,1595202607
296,296,lithakia,37.72,20.83,0,8.05,60,73.40,GR,1595202609


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = new_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
21,ajdabiya,LY,30.76,20.23,
68,boysun,UZ,38.21,67.20,
85,surt,LY,31.21,16.59,
95,kallithea,GR,37.95,23.70,
103,rio brilhante,BR,-21.80,-54.55,
118,gornoye loo,RU,43.71,39.60,
151,manavgat,TR,36.79,31.44,
228,marsa matruh,EG,31.35,27.25,
283,hyeres,FR,43.12,6.12,
296,lithakia,GR,37.72,20.83,


In [29]:
#set params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
#Iteration
for index, row in hotel_df.iterrows():
    lat= row["Lat"]
    lng= row["Lng"]
    
    params["location"] = f"{lat}, {lng}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_req = requests.get(url, params=params)
    
    hotel_req = hotel_req.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_req["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
21,ajdabiya,LY,30.76,20.23,فندق التوجان
68,boysun,UZ,38.21,67.20,Gaza Hotel
85,surt,LY,31.21,16.59,City Hotel
95,kallithea,GR,37.95,23.70,Central Hotel
103,rio brilhante,BR,-21.80,-54.55,Hotel Cattani
118,gornoye loo,RU,43.71,39.60,«ВатерЛоо» гостиница
151,manavgat,TR,36.79,31.44,Lake & River Side Hotel & Spa
228,marsa matruh,EG,31.35,27.25,Beau Site
283,hyeres,FR,43.12,6.12,ibis budget Hyères Centre-Ville
296,lithakia,GR,37.72,20.83,Zante Plaza Hotel & Apartments


In [30]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))